In [1]:
!aws s3 cp s3://msan694-group/final_nba.csv final_nba.csv
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline

download: s3://msan694-group/final_nba.csv to ./final_nba.csv


In [2]:
rdd_vec = sc.textFile("final_nba.csv")
rdd_vec.take(5)

NameError: name 'sc' is not defined

In [16]:
rdd_vec = sc.textFile("final_nba.csv")
header = rdd_vec.first() #extract header
data = rdd_vec.filter(lambda row: row not in header) #filter out the header!
new_rdd = data.map(lambda line: line.split(',')) # split the wide vector by ","
#turn to Vectors.dense (with label out front)
split_rdd = new_rdd.map(lambda line: (float(line[0]), Vectors.dense([float(c) for c in line[1:len(line)]]))) 
#split_rdd = new_rdd.map(lambda line: (float(line[0]), Vectors.dense([float(c) for c in line[1:25000]]))) 
# Create the DataFrame from the collected RDD
full_df = sqlContext.createDataFrame(split_rdd.collect(), ["label", "features"])
full_df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|[1.0,2.0,1.9,7.7,...|
|  0.0|[2.0,0.0,0.8,28.2...|
|  0.0|[3.0,3.0,2.7,10.1...|
|  0.0|[4.0,2.0,1.9,17.2...|
|  0.0|[5.0,2.0,2.7,3.7,...|
|  0.0|[6.0,2.0,4.4,18.4...|
|  0.0|[7.0,11.0,9.0,20....|
|  1.0|[8.0,3.0,2.5,3.5,...|
|  0.0|[9.0,0.0,0.8,24.6...|
|  0.0|[1.0,0.0,1.1,22.4...|
|  0.0|[2.0,8.0,7.5,24.5...|
|  1.0|[3.0,14.0,11.9,14...|
|  1.0|[4.0,2.0,2.9,5.9,...|
|  0.0|[1.0,0.0,0.8,26.4...|
|  0.0|[1.0,0.0,0.5,22.8...|
|  1.0|[2.0,3.0,2.7,24.7...|
|  0.0|[3.0,6.0,5.1,25.0...|
|  0.0|[4.0,1.0,0.9,25.6...|
|  1.0|[5.0,0.0,1.2,24.2...|
|  0.0|[1.0,2.0,2.2,25.4...|
+-----+--------------------+
only showing top 20 rows



In [17]:
%%time
(trainingData, testData) = full_df.randomSplit([0.7, 0.3])
gbt = GBTClassifier(maxIter=2, maxDepth=2, labelCol="label")
pipeline = Pipeline(stages=[gbt])
model = pipeline.fit(trainingData)

CPU times: user 14.8 ms, sys: 4.69 ms, total: 19.4 ms
Wall time: 3.01 s


In [20]:
# Make predictions.
predictions = model.transform(testData)
predictions.select("prediction", "label", "features").show(20)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[1.0,0.0,-0.3,18....|
|       1.0|  0.0|[1.0,0.0,0.0,0.1,...|
|       1.0|  0.0|[1.0,0.0,0.0,1.5,...|
|       0.0|  0.0|[1.0,0.0,0.4,22.2...|
|       0.0|  0.0|[1.0,0.0,0.5,22.8...|
|       0.0|  0.0|[1.0,0.0,0.5,25.6...|
|       0.0|  0.0|[1.0,0.0,0.6,5.0,...|
|       0.0|  0.0|[1.0,0.0,0.6,6.6,...|
|       0.0|  0.0|[1.0,0.0,0.6,7.6,...|
|       0.0|  0.0|[1.0,0.0,0.6,15.3...|
|       0.0|  0.0|[1.0,0.0,0.6,17.3...|
|       0.0|  0.0|[1.0,0.0,0.6,24.3...|
|       0.0|  0.0|[1.0,0.0,0.6,25.1...|
|       1.0|  0.0|[1.0,0.0,0.7,2.9,...|
|       0.0|  0.0|[1.0,0.0,0.7,6.3,...|
|       0.0|  0.0|[1.0,0.0,0.7,6.7,...|
|       0.0|  0.0|[1.0,0.0,0.7,17.1...|
|       0.0|  0.0|[1.0,0.0,0.7,19.8...|
|       0.0|  0.0|[1.0,0.0,0.7,20.3...|
|       0.0|  0.0|[1.0,0.0,0.7,22.8...|
+----------+-----+--------------------+
only showing top 20 rows



In [21]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % (accuracy*100))

Accuracy = 58.127
